In [50]:
from faker import Faker
import csv
import random
from datetime import datetime, timedelta
import string

p = Faker()

In [51]:
ticketcount = 40000
airplanecount = 28000
reservationcount = 10000
usercount = 100000
passengercount = 40000
flightcount = 800
checkscount = 100000

In [52]:
airport_names = []
airlines_data = []

csv_file_path = 'Airline Dataset Updated - v2.csv'
with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)

    headers = next(csv_reader)
    airport_name_index = headers.index('Airport Name')
    departure_date_index = headers.index('Departure Date')
    arrival_airport_index = headers.index('Arrival Airport')
    for row in csv_reader:
        airport_names.append(row[airport_name_index])


csv_file_path = 'airlines.csv'
with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)

    headers = next(csv_reader)

    airline = headers.index('Name')
    iata = headers.index('IATA')

    for row in csv_reader:
        if row[iata].strip() == "":
            continue
        airlines_data.append( (row[airline], row[iata]) )

emails = { p.email() for i in range(1000) }

In [53]:
def generate_passport_id():
    while True:
        passport_id = "".join(random.choice(string.ascii_uppercase + string.digits) for _ in range(8))
        yield passport_id

def generate_phone_number():
    while True:
        phone_number = "0" + "".join(str(random.randint(0, 9)) for _ in range(9))
        yield phone_number


phone_number_generator = generate_phone_number()
passport_id_generator = generate_passport_id()

phones = list( { next(phone_number_generator) for i in range(passengercount*10)} )
passportid = list( {next(passport_id_generator) for i in range(passengercount)} ) #70000

In [54]:
print(len(phones), len(passportid))
aircraft_models = ["A320", "B738", "A321", "A319", "B77W", "B737", "A333", "A332", "B739", "DH8D", "E190", "E170", "B763", "B772", "B788", "B752", "A388", "B744", "CRJ9", "AT76", "B789", "AT75", "B77L", "C172"]

399910 40000


In [55]:
fctypes = [("Gold", 30), ("Silver", 20), ("Explorer", 10), ("Platinum", 40)]

In [56]:
def gen_seats(model):
    x = random.randint(100, 300)
    return (
        int(x * 1/2),
        int(x * 1/6),
        int(x * 1/6),
        int(x * 1/6),
        random.randint(500, 1000),
        model 
    )

AirplaneModelsEntity = [
    gen_seats(model)
    for model in aircraft_models
]

AirplanesEntity = [ (
        i +1 , 
        random.choice(airlines_data)[0],     
        random.choice(aircraft_models), 
    ) for i in range(airplanecount)
]


In [57]:
def genFlight(i):
    x = p.date_time_this_decade()

    return     (
        str(random.choice(airlines_data)[1]) + str(i + 1),
        x.strftime("%Y-%m-%d %H:%M:%S"), #Arrival TIME
        (x - timedelta(days=random.randint(0, 24))).strftime("%Y-%m-%d %H:%M:%S"), #Departure TIME
        random.choice(airport_names), #Destination
        random.choice(airport_names), #Departure
        random.randint(1, airplanecount)
    )

FlightsEntity = [
    genFlight(i)
    for i in range(flightcount)
]

In [58]:
UserEntity = [
    (mail,
    p.first_name(),
    p.last_name(),
    p.date_of_birth(minimum_age=18, maximum_age=65).strftime("%Y-%m-%d"),
    p.password())
    for mail in emails
]

In [59]:

PassengerEntity = [
    (
        int((i+1)*3), #CIN
        p.date_of_birth(minimum_age=18, maximum_age=80).strftime("%Y-%m-%d"),
        pa, #PASSPORT
        random.choice(["NULL", phones[i], phones[i]]),
        p.first_name(),
        p.last_name(),
        random.choice([(random.choice(fctypes), p.uuid4()), "NULL"])  
    )
    for i, pa in enumerate( passportid )
]

In [60]:
emails = list(emails)

def reserve(i):
    x = p.date_time_this_decade()
    return    (
        i+1, 
        x.strftime("%Y-%m-%d %H:%M:%S"),
        random.choice( ["NULL", (x + timedelta(hours=random.randint(0, 24))).strftime("%Y-%m-%d %H:%M:%S")]),
        random.choice(UserEntity)[0],
    )

ReservationsENTITY = [
    
    reserve(i)

    for i in range(reservationcount)
]

In [61]:

def genticket(i):
    x = random.choice(PassengerEntity)
    y = "NULL" if x[6] == "NULL" else x[6][0][0]
    return (i+1,
        random.randint(100, 300),
        random.randint(100, 1300),
        x[2],
        random.choice(ReservationsENTITY)[0],
        random.choice(FlightsEntity)[0],
        y,
        random.choice(["Y", "J", "FC", "Y+"]),
    )

TicketENTITY = [
    genticket(i)
    for i in range(ticketcount)
]




In [62]:
ChecksEntity = {
    (
        random.choice(UserEntity)[0],
        random.choice(FlightsEntity)[0],
    )
    for i in range(checkscount) #400000
}

In [63]:
def write_to_csv(filename, data):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerows(data)

def populatefirst():
    data = [("fctype", "reduction")]
    for i in fctypes: 
        data.append(i)
    write_to_csv('t/FidelityCard.csv', data)

    data = [("registrationNumber", "airline", "model")]
    for i in AirplanesEntity:
        data.append(i)
    write_to_csv('t/airplane.csv', data)

    data = [("economySeats", "premiumEconomySeats", "businessClassSeats", "firstClassSeats", "maxweight", "model")]
    for i in AirplaneModelsEntity: 
        data.append(i)
    write_to_csv('t/airplanemodel.csv', data)

    data = [("fid", "arrivalTime", "departureTime", "destination", "departure", "registrationNumber")]
    for i in FlightsEntity: 
        data.append(i)
    write_to_csv('t/flight.csv', data)

    data = [("uemail", "ufirstName", "ulastName", "ubirthDate", "passwordHash")]
    for i in UserEntity: 
        data.append(i)
    write_to_csv('t/user_.csv', data)


    data_passengercard = [("fctype", "fcid")]
    data_passengerinfos = [("cin", "pbirthDate", "passportID", "phoneNumber", "pfirstName", "plastName", "fcid")]
    
    for i in PassengerEntity: 
        l = i
        if i[6] != "NULL":
            row_passengercard = (i[6][0][0], i[6][1])
            l = (i[0], i[1], i[2], i[3], i[4], i[5], i[6][1]) 
            data_passengercard.append(row_passengercard)
            
        data_passengerinfos.append(l)
    
    write_to_csv('t/passengercard.csv', data_passengercard)
    write_to_csv('t/passenger.csv', data_passengerinfos)


    data = [("email", "fid")]
    for i in ChecksEntity: 
        data.append(i)
    write_to_csv('t/checks.csv', data)


    data_reservation = [("rid", "dateReservation", "dateConfirmation", "email")]
    data_ticket = [("tid", "seatNumber", "price", "passportID", "rid", "fid", "fctype", "class")]

    for i in ReservationsENTITY: 
        data_reservation.append(i)

    for i in TicketENTITY: 
        data_ticket.append(i)

    write_to_csv('t/reservation.csv', data_reservation)
    write_to_csv('t/ticket.csv', data_ticket)

populatefirst()

In [64]:
"""def csv_to_sql_insert(csv_file):
    table_name = csv_file.replace(".csv", "")

    with open("t/" + csv_file, 'r', newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        columns = csv_reader.fieldnames
        
        x = str(tuple(columns)).replace("'", "") 
        insert_queries = [f"INSERT INTO {table_name} {x} VALUES"]
        
        for row in csv_reader:
            values = [f"{row[column]}" if isinstance(row[column], str) else str(row[column]) for column in columns]
            y = str(tuple(values)).replace("'NULL'", "NULL")
            insert_query = f'{y},'
            insert_queries.append(insert_query)

        insert_queries[-1] = insert_queries[-1][0:-1]
        insert_queries.append(";")

    return insert_queries 
"""

def csv_to_sql_insert(csv_file):
    table_name = csv_file.replace(".csv", "")

    with open("t/" + csv_file, 'r', newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        columns = csv_reader.fieldnames
        
        x = str(tuple(columns)).replace("'", "") 
        insert_queries = []
        
        for row in csv_reader:
            values = [f"{row[column]}" if isinstance(row[column], str) else str(row[column]) for column in columns]
            y = str(tuple(values)).replace("'NULL'", "NULL")
            insert_query = f'INSERT INTO {table_name} {x} VALUES {y};'
            insert_queries.append(insert_query)

    return insert_queries 


tables = ["airplanemodel", "airplane", "FidelityCard", "flight", "passengercard", "passenger", "user_", "checks", "reservation", "ticket"]
insert_queries = []

for tablename in tables:
    insert_queries += csv_to_sql_insert(f"{tablename}.csv")


In [ ]:
with open("dataloader.sql", "w+", encoding='utf-8') as file:
    for query in insert_queries:
        file.write(query + "\n")
